In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan  3 15:53:07 2020

@author: whitma01
"""
import cv2
import numpy as np
from PIL import Image
import sklearn.metrics as sk
from tqdm import tqdm
import os
import SimpleITK as sitk
import glob, os


def remove_small_regions(m_mask, threshold): 

    img = sitk.GetImageFromArray(m_mask)
    cca = sitk.ConnectedComponentImageFilter()
    cca_image = cca.Execute(img)
    stats = sitk.LabelShapeStatisticsImageFilter()                                                                                                                                     
    stats.Execute(cca_image)

    relabelMap =  { i : 0 for i in stats.GetLabels() if stats.GetNumberOfPixels(i) < threshold }
    output = sitk.ChangeLabel(cca_image, changeMap=relabelMap)

    del img
    del cca
    del cca_image
    del stats

    m_mask_inverse = sitk.GetArrayFromImage(output)
    m_mask_inverse = m_mask_inverse != 0
    return m_mask_inverse

#https://biomedical-engineering-online.biomedcentral.com/articles/10.1186/s12938-018-0518-0


exclude = ["685039_(1.00,17661,41360,19968,2561)",
    "685039_(1.00,7007,741,9216,1024)",
    "685039_(1.00,81054,80,2561,3585)",
    "685040_(1.00,12444,41415,17920,1536)",
    "685042_(1.00,14,40462,17409,1025)",
    "685042_(1.00,22058,190,12800,1024)",
    "688058_(1.00,15734,32438,3072,2560)",
    "688059_(1.00,22312,137,2560,2049)",
    "688059_(1.00,30214,40324,6656,1537)",
    "688060_(1.00,2000,40,9728,2048)",
    "688061_(1.00,11308,19,6656,513)",
    "688061_(1.00,3687,38203,10240,3072)",
    "688062_(1.00,2057,40517,2561,1537)",
    "688062_(1.00,29137,99,3072,1537)",
    "688063_(1.00,144,39239,8192,1536)",
    "688063_(1.00,59467,8417,1024,3584)",
    "688064_(1.00,36,16442,5120,2560)",
    "688065_(1.00,10100,27089,1024,1024)",
    "688065_(1.00,17683,18489,1024,2048)",
    "688065_(1.00,70846,38229,7168,2560)",
    "688066_(1.00,19682,78,2048,2561)",
    "688066_(1.00,37781,26018,1025,1025)",
    "688067_(1.00,36205,37462,2048,1024)",
    "688067_(1.00,36818,33694,1025,1025)",
    "688067_(1.00,6355,8220,3072,2560)",
    "688068_(1.00,29035,40904,3072,2560)",
    "688068_(1.00,68661,18472,3072,1024)",
    "688069_(1.00,29,3205,513,4096)",
    "688070_(1.00,39203,33602,2048,1024)",
    "688070_(1.00,42838,10,1536,2560)",
    "688071_(1.00,48355,40346,3584,1536)",
    "688071_(1.00,68742,11843,1536,1536)",
    "688072_(1.00,167,29474,4608,1536)",
    "688072_(1.00,37596,7981,2560,2048)",
    "688072_(1.00,43334,28816,2048,2560)",
    "688072_(1.00,51839,26482,1536,1536)",
    "688073_(1.00,41390,5217,1536,1536)",
    "688073_(1.00,42068,31365,2048,4096)",
    "688073_(1.00,6443,2178,1025,4097)",
    "688074_(1.00,5212,37585,3584,1024)",
    "688074_(1.00,6985,33897,1536,1024)",
    "688075_(1.00,52,24706,3585,2048)",
    "688076_(1.00,2373,8304,1024,2048)",
    "688076_(1.00,40252,19178,2048,3072)",
    "688076_(1.00,998,13583,1025,1025)",
    "688077_(1.00,19589,3339,1024,1024)",
    "688077_(1.00,43727,34903,2560,512)",
    "688077_(1.00,464,357,3072,1024)",
    "688077_(1.00,68948,32390,1536,512)",
    "688078_(1.00,44049,40715,5120,2560)",
    "688078_(1.00,63258,7450,1536,1024)",
    "688078_(1.00,65548,21132,2049,1025)",
    "688079_(1.00,23762,5020,1024,2048)",
    "688079_(1.00,4134,17635,1024,2560)",
    "688079_(1.00,46337,3527,1024,1536)",
    "688080_(1.00,2964,22951,2048,1536)",
    "688080_(1.00,36554,385,1024,1536)",
    "688081_(1.00,23205,154,8192,512)",
    "688081_(1.00,5887,36485,2048,2048)",
    "688082_(1.00,34697,37863,4608,3584)",
    "688083_(1.00,33758,10111,2048,1536)",
    "688084_(1.00,31634,38140,3072,2560)",
    "688135_(1.00,15442,3051,3072,1024)",
    "688135_(1.00,55423,30392,1024,2048)",
    "688085_(1.00,13134,1089,14848,4096)",]


names = ['684913','684914','684916','685039','685040','685041','685042','685281']
directory_true = 'masksMISSION_corrected/'
directory_pred = 'Inference_MISSION/'


# names = ['688058','688059','688060','688061','688062','688063','688064','688065',
#           '688066','688067','688068','688069','688070','688071','688072','688073','688074',
#           '688075','688076','688077','688078','688079','688080','688081','688082','688083','688084','688135']
# directory_true = 'masksBEHOLD_corrected/'
# directory_pred = 'Inference_BEHOLD/'


tissues = ['Tumor','Benign','Adipose','Stroma','Macrophages','Lymphocytes','Background','Nerve','Muscle','Vessel']


for j, tissue in enumerate(tissues):
    print(tissue)
    data_writer = open("evaluation/validate_" + tissue + "_MISSION_together.csv", "w")
    # data_writer = open("evaluation/validate_" + tissue + "_BEHOLD_together.csv", "w")
    
    data_writer.write("DICE, JACCARD\n")
    data_writer.flush()
    os.fsync(data_writer.fileno())

    numerator_DICE_total = 0
    denominator_DICE_total = 0
    numerator_JACCARD_total = 0
    denominator_JACCARD_total = 0

    for i, name in enumerate(names):
        # print(name)
        # print() 

        #os.chdir(directory_pred)
        for file in glob.glob(directory_pred + "*.png"):

            if name in file and tissue in file:
                imageID = os.path.basename(file[:-4])
                patientID = imageID.split("_")
                imageID2 = patientID[0] + "_" + patientID[1]

                if imageID2 in exclude:
                    print("excluding " + imageID2)
                else:

                    numerator_DICE = 0
                    denominator_DICE = 0
                    numerator_JACCARD = 0
                    denominator_JACCARD = 0

                    imageID = os.path.basename(file[:-4])
                    imageID2 = imageID.split("_")

                    filename_predicted = directory_pred + os.path.basename(file)
                    # print('reading predicted mask ' + filename_predicted)
                    mask_predicted = cv2.imread(filename_predicted, 0)

                    filename_true = directory_true + imageID2[0] + "_" + imageID2[2] + "_" + imageID2[1] + "-mask.png"
                    if tissue == 'Vessel':
                        filename_true = directory_true + imageID2[0] + "_vessel_" + imageID2[1] + "-mask.png"
                    # print('reading true mask ' + filename_true)
                    mask_true = cv2.imread(filename_true, 0)

                    
                    threshold_predicted = mask_predicted.max() / 2
                    threshold_true = mask_true.max() / 2
                    mask_predicted_bin = mask_predicted > threshold_predicted
                    mask_true_bin = mask_true > threshold_true


                    numerator_DICE = 2. * (mask_predicted_bin & mask_true_bin).sum()
                    denominator_DICE = (mask_predicted_bin.sum() + mask_true_bin.sum())
                    # print('numerator_DICE: {:.2f}'.format(numerator_DICE))
                    # print('denominator_DICE: {:.2f}'.format(denominator_DICE))

                    numerator_JACCARD = (mask_predicted_bin & mask_true_bin).sum()
                    denominator_JACCARD = (mask_predicted_bin | mask_true_bin).sum()
                    #denominator_JACCARD = mask_predicted_bin.sum() + mask_true_bin.sum() - (mask_predicted_bin & mask_true_bin).sum()
                    # print('numerator_JACCARD: {:.2f}'.format(numerator_JACCARD))
                    # print('denominator_JACCARD: {:.2f}'.format(denominator_JACCARD))

                    numerator_DICE_total += numerator_DICE
                    denominator_DICE_total += denominator_DICE
                    numerator_JACCARD_total += numerator_JACCARD
                    denominator_JACCARD_total += denominator_JACCARD


    print('numerator_DICE_total: {:.2f}'.format(numerator_DICE_total))
    print('denominator_DICE_total: {:.2f}'.format(denominator_DICE_total))
    print('numerator_JACCARD_total: {:.2f}'.format(numerator_JACCARD_total))
    print('denominator_JACCARD_total: {:.2f}'.format(denominator_JACCARD_total))

    if(denominator_JACCARD_total == 0 or denominator_DICE_total == 0):
        DICE = 0;
        JACCARD = 0;
    else:
        DICE = float(numerator_DICE_total) / float(denominator_DICE_total)
        print('DICE: {:.3f}'.format(DICE))
        JACCARD = float(numerator_JACCARD_total) / float(denominator_JACCARD_total)
        print('JACCARD: {:.3f}'.format(JACCARD))
        print()

    data_writer.write(str(DICE) + "," + str(JACCARD) + "\n")
    data_writer.flush()
    os.fsync(data_writer.fileno())

    data_writer.close()

Tumor
excluding 685039_(1.00,81054,80,2561,3585)
excluding 685039_(1.00,17661,41360,19968,2561)
excluding 685039_(1.00,7007,741,9216,1024)
excluding 685040_(1.00,12444,41415,17920,1536)
excluding 685042_(1.00,22058,190,12800,1024)
excluding 685042_(1.00,14,40462,17409,1025)
numerator_DICE_total: 335275622.00
denominator_DICE_total: 411258528.00
numerator_JACCARD_total: 167637811.00
denominator_JACCARD_total: 243620717.00
DICE: 0.815
JACCARD: 0.688

Benign
excluding 685039_(1.00,81054,80,2561,3585)
excluding 685039_(1.00,17661,41360,19968,2561)
excluding 685039_(1.00,7007,741,9216,1024)
excluding 685040_(1.00,12444,41415,17920,1536)
excluding 685042_(1.00,22058,190,12800,1024)
excluding 685042_(1.00,14,40462,17409,1025)
numerator_DICE_total: 33424742.00
denominator_DICE_total: 60140252.00
numerator_JACCARD_total: 16712371.00
denominator_JACCARD_total: 43427881.00
DICE: 0.556
JACCARD: 0.385

Adipose
excluding 685039_(1.00,81054,80,2561,3585)
excluding 685039_(1.00,17661,41360,19968,2561)